In [1]:
import os
import pandas as pd
import numpy as np
from typing import Dict, List
import importlib
from pathlib import Path
import pprint as pp
from logging import DEBUG
from torch.utils.data import DataLoader
import torch

In [2]:
import sys

sys.path.append("..")
from src.gradepred_data import GradePredictionDataset, GradePredictionCollator
from src.util import set_logger, load_model

### データセット -> Collator までの可視化・分析

In [3]:
import sys

logger = set_logger(name="LLM_test", level=DEBUG)

2025-06-15 18:15:27,558 : LLM_test : INFO : 58 : Test_message


In [4]:
BASE_PATH = Path.cwd().parent
DATA_PATH = BASE_PATH / "data"


In [5]:
# data = GradePredictionDataset(dataset_path=DATA_PATH, logger=logger)
Q1_data = GradePredictionDataset(
    dataset_path=DATA_PATH,
    question_filter=[1],
    logger=logger,
)

2025-06-15 18:15:30,781 : LLM_test : INFO : 142 : Dataset init done. users=377, qs=[1], concat=False
2025-06-15 18:15:30,784 : LLM_test : INFO : 214 : Dataset build done: total=377, 


In [6]:
pp.pprint(Q1_data.__len__())
pp.pprint(Q1_data.__getitem__(0).keys())
print("Elements:")
print("===========================")
for i in range(5):
    print(f"Element {i}:")
    pp.pprint(str(Q1_data.__getitem__(i)))
    print("===========================")

377
dict_keys(['userid', 'labels', 'grades', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15'])
Elements:
Element 0:
("{'userid': 'C-2021-1_U1', 'labels': 4, 'grades': 'F', 'L1': {'Q1': "
 "'情報伝達には声、モールス信号など様々な形があり、人類の進化と共により高度に精度が高められてきた。'}, 'L2': {'Q1': "
 "'情報源符号化とは、情報を黒と白の○で表すことで、複合とは符号化によって得られたものを元に戻す作業のことである。それらの過程の中で、平均符号語長を短くすることが目指される。そこで一意符号可能性と瞬時復号可能性を持つ語頭符号を用いる。 "
 "エントロピーとは平均符号語長の下限のことである。'}, 'L3': {'Q1': "
 "'生起確率pの事象の生起を知ったことによる曖昧さの減少量は情報量と一致し、生起確率pが高いほど情報量は現状しまた逆もしかりである。またエントロピーは情報量の期待値と一致する。相互情報量とは、ある事象2つが相互に関連して生起した場合に得られる情報量のことである。'}, "
 "'L4': {'Q1': "
 "'通信の際に符号の誤りを検出・訂正する仕組みについて学んだ。符号の誤りは、符号語同士がs+1以上離れていればs個検出でき、2t+1離れていればt個訂正することができる。また、その距離をハミング距離と言う。'}, "
 "'L5': {'Q1': "
 "'コンピューターサイエンスは計算機科学であるともされ、全ての科学技術分野の基盤となりうるものである。コンピューター上で行われている全てのことは「計算」であり、コンピューターサイエンスはそれを制御・理解するための科学である。 "
 "アルゴリズムとは、関数の計算方法を示す手続きのことを言い、プログラムとはアルゴリズムをコンピューターが分かる言葉で記述したものである。アルゴリズムを上手く考えることができれば、「偽コイン発見」の問題のように作業を効率化・最適化することができる。大きな数の因数分

In [14]:
eval_dataset = GradePredictionDataset(
    dataset_path=DATA_PATH,
    question_filter=[1, 2, 3, 4, 5],
    concatenate=False,
    mode="valid",
    division=True,
)

In [15]:
print("Eval dataset:")
pp.pprint(eval_dataset.__len__())
print("Eval dataset elements:")
for i in range(5):
    print(f"Element {i}:")
    pp.pprint(str(eval_dataset.__getitem__(i)))
    print("===========================")

Eval dataset:
5700
Eval dataset elements:
Element 0:
("{'userid': 'C-2021-2_U128', 'labels': 1, 'grades': 'B', 'input_text': "
 "'L1-Q1: "
 "本日の講義において「情報」は人間のコミュニケーションを円滑に行う道具として役に立っていることを学んだ。例えば、音声を伝えることに関しては私たちが小さいころしていた糸電話や昭和時代に使用されていた音をアナログ電気信号として伝えるアナログ式電話そして時は流れ私たちが使用しているデジタル式電話など情報を伝える方法は変化を遂げている。また情報通信機器では、文字や写真などの情報が0と1の並びとして表されている。また情報を伝えるうえで私たちが快適に情報を伝えるために情報源符号化、通信路符号化、暗号化などの機能もある。また情報を伝える手段を幅広く見ていくと遠くに伝えるためには大昔はほら貝やたいこなどを用いて合図を送り、7千年前~5千年前で世界四大文明が文字を発明した。日本の江戸時代は合図のほか伝書バトや飛脚で文章を送る方法や松明通信や腕木信号や手旗信号やモールス電信やバケツリレー式で情報を伝える手段は時が経つにつれて増加してきたことが分かる。増加してきたことが分かる。そして、私たちの時代には、携帯電話やファックスなどが普及する。このように情報は人々とのコミュニケーションを円滑に行う道具であり、またその情報を伝える方法も場所や時代の変化に伴い増加している。'}")
Element 1:
("{'userid': 'C-2021-2_U128', 'labels': 1, 'grades': 'B', 'input_text': "
 "'L1-Q2: "
 "今回の講義において、情報を伝える方法がとても多いことに驚きました。、モールス信号はスタジオジブリの「崖に上のポニョ」で知っていましたが、まだまだ自分の未知なるものがあることが分かりました。また主観的評価になってしまいますが、情報を伝える手段がどのようなものかを理解できたかと思います。これからも情報に関しての理解を深めていきたいと思います。'}")
Element 2:
("{'userid': 'C-2021-2_U128', 'labels': 1

In [ ]:
train_dataset = GradePredictionDataset(
    dataset_path=DATA_PATH,
    question_filter=[1],
    concatenate=True,
    mode="train",
)
eval_dataset = GradePredictionDataset(
    dataset_path=DATA_PATH,
    question_filter=[1],
    concatenate=True,
    mode="valid",
)

In [ ]:
print("Train dataset:")
pp.pprint(train_dataset.__len__())
print("Eval dataset:")
pp.pprint(eval_dataset.__len__())
print("Train dataset elements:")
for i in range(3):
    print(f"Element {i}:")
    pp.pprint(str(train_dataset.__getitem__(i)))
    print("===========================")

Train dataset:
301
Eval dataset:
76
Train dataset elements:
Element 0:
("{'userid': 'C-2022-1_U80', 'labels': 2, 'grades': 'C', 'input_text': "
 "'L01-Q1: 情報伝達の歴史について "
 '〇or✖のようなシンプルな情報伝達から取り決めによってより具体的に事物を伝えられるようになり、機械の発達で高速化した。\\nL02-Q1: '
 '語頭符号について\\nL03-Q1: ハミング距離の理論によってビットの誤りを修正できる。\\nL04-Q1: '
 '情報漏洩を防ぐために暗号化は大事\\nL05-Q1: アルゴリズムについて アルゴリズムとは問題を解決するための手順や計算方法のこと '
 'アルゴリズム体操もおなじ手順を繰り返している!!!\\nL06-Q1: より少ない手順で符号を並び替える方法とその大切さ\\nL07-Q1: '
 'ソートについて\\nL08-Q1: 情報の検索について\\nL09-Q1: データの整理と生活とコンピュータの共同、かかわり\\nL10-Q1: '
 '言語などの構造化されていないデータの処理に行いて\\nL11-Q1: 情報の様々な抽出方法 '
 '情報をオープンデータにするためには、クリエイティブ・コモンズ・ライセンスをつけて公開しなければいけない。\\nL12-Q1: '
 'データの解析において様々な距離の種類があり、用途が異なる 距離と類似度は反対の概念\\nL13-Q1: '
 'データを数値化、可視化することの利点と表現方法の注意 3Dパイチャートはあまりよくない\\nL14-Q1: さまざまなフィルタ処理で画質を良くしている '
 "画像処理と芸工で学ぶ技術は密接に関係している\\nL15-Q1: NaN'}")
Element 1:
("{'userid': 'C-2021-1_U81', 'labels': 3, 'grades': 'D', 'input_text': "
 "'L01-Q1: イントロダクション 情報通信方法\\nL02-Q1: NaN\\nL03-Q1: NaN\\nL04-Q1: "
 'NaN\\nL05-Q1: 効率のよいアルゴリズ

In [ ]:
model, tokenizer = load_model("elyza/Llama-3-ELYZA-JP-8B", if_ZeRO=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[info] Loading processor (use_fast=True)...
✅ Model and processor loaded successfully!


In [ ]:
train_logger = set_logger(name="CollateTrain", level=DEBUG)
eval_logger = set_logger(name="CollateEval", level=DEBUG)

2025-06-15 15:10:04,850 : CollateTrain : INFO : 58 : Test_message
2025-06-15 15:10:04,851 : CollateEval : INFO : 58 : Test_message


In [ ]:
train_collator = GradePredictionCollator(
    tokenizer,
    max_tokens=4096,
    include_target=True,
    logger=train_logger,
    question_filter=[1],
)
eval_collator = GradePredictionCollator(
    tokenizer,
    max_tokens=4096,
    include_target=False,
    logger=eval_logger,
    question_filter=[1],
)

2025-06-15 15:10:04,920 : CollateTrain : INFO : 431 : Questions: [1], Q1:今日の内容を自分なりの言葉で説明してみてください

2025-06-15 15:10:04,922 : CollateTrain : INFO : 445 : preamble set:
あなたは大学の教授であり、学生の成績を決定する役割を担っています。以下に示す学生の講義後アンケートを読み、成績を高い順に A、B、C、D、F のいずれかに分類してください。
成績は出力例のような形式で出力してください。
入力文のL は講義回、Q は質問番号を示します（例: L1-Q1）。
アンケートの質問文は、
Q1:今日の内容を自分なりの言葉で説明してみてください
です。回答が NaN の場合は未回答です。
出力には A、B、C、D、F のいずれかを含めてください。
出力例:
この学生の成績は、Aです。
アンケート内容：

2025-06-15 15:10:04,923 : CollateEval : INFO : 431 : Questions: [1], Q1:今日の内容を自分なりの言葉で説明してみてください

2025-06-15 15:10:04,924 : CollateEval : INFO : 445 : preamble set:
あなたは大学の教授であり、学生の成績を決定する役割を担っています。以下に示す学生の講義後アンケートを読み、成績を高い順に A、B、C、D、F のいずれかに分類してください。
成績は出力例のような形式で出力してください。
入力文のL は講義回、Q は質問番号を示します（例: L1-Q1）。
アンケートの質問文は、
Q1:今日の内容を自分なりの言葉で説明してみてください
です。回答が NaN の場合は未回答です。
出力には A、B、C、D、F のいずれかを含めてください。
出力例:
この学生の成績は、Aです。
アンケート内容：



In [ ]:
# collatorの動作を確認
example_loader = DataLoader(
    train_dataset, collate_fn=train_collator, batch_size=4, shuffle=True
)
batch = next(iter(example_loader))
logger.info(
    f"Batch keys: {batch.keys()}"
)  # -> Batch keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
for k, v in batch.items():
    logger.info(
        f"Batch key: {k}, value shape: {v.shape if isinstance(v, torch.Tensor) else type(v)}"
    )
# batch内のinput_idsをでコードしてプロンプトを確認
input_text = tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=True)
logger.info(f"Input prompt: {input_text}")
logger.debug(f"\n{batch}")

2025-06-15 15:10:05,036 : CollateTrain : DEBUG : 479 : prompt sample:
[INST] あなたは大学の教授であり、学生の成績を決定する役割を担っています。以下に示す学生の講義後アンケートを読み、成績を高い順に A、B、C、D、F のいずれかに分類してください。
成績は出力例のような形式で出力してください。
入力文のL は講義回、Q は質問番号を示します（例: L1-Q1）。
アンケートの質問文は、
Q1:今日の内容を自分なりの言葉で説明してみてください
です。回答が NaN の場合は未回答です。
出力には A、B、C、D、F のいずれかを含めてください。
出力例:
この学生の成績は、Aです。
アンケート内容：
L01-Q1: これまで高等学校では情報教育において多くの改善がなされてきたがサイエンスとしての情報がないがしろにされている。大学ではこれまでと違い理系科目として「情報科学」を学んでいく。 情報伝達には情報源符号化、通信路符号化、暗号化がある。アナログからデジタルまで情報伝達の手段は大昔から変遷をたどってきたが、その根本にあるものはいつもかわらない。
L02-Q1: 平均符号語長の出来るだけ短い符号を設計することを目的として情報源符号化を行う。ただし一
2025-06-15 15:10:05,037 : CollateTrain : DEBUG : 480 : grade sample:  この学生の成績は、Bです。
2025-06-15 15:10:05,050 : CollateTrain : DEBUG : 499 : Tokenization Done
2025-06-15 15:10:05,052 : CollateTrain : DEBUG : 545 : Collating Done
2025-06-15 15:10:05,052 : CollateTrain : DEBUG : 547 : Collate: 4 samples, max_tokens=4096
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` 

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
# batchのサンプルから，input_idsとlabelsを取り出して，全体を表示
# テンソルの全体を表示するために、`torch.set_printoptions`を使用
torch.set_printoptions(threshold=10000, edgeitems=30)
ids = 0
input_ids = batch["input_ids"][ids]
labels = batch["labels"][ids]

pp.pprint(f"Input IDs: {input_ids}")
pp.pprint(f"Labels: {labels}")

# 戻す
torch.set_printoptions(threshold=1000, edgeitems=3)

('Input IDs: tensor([128000,     27,     91,   9125,     91,     29,  30591, '
 '112568,  15682,\n'
 '        102667,  16144, 116891, 116787,   5486, 106718,  16144,  13153, '
 '116486,\n'
 '         30512, 122074,  54926, 107734, 111017,  30512, 106529,  76947,  '
 '61689,\n'
 '          1811, 128009, 128006,    882, 128007,  88852,  20230,  20379,  '
 '17663,\n'
 '        106718,  16144, 114341, 104577,  74482, 113164, 102663,  84477,  '
 '30512,\n'
 '         98499,  64121,   5486,  13153, 116486,  30512, 126574, 110480,  '
 '20230,\n'
 '           362,   5486,     33,   5486,     34,   5486,     35,   5486,     '
 '37,\n'
 '         97718,  16995, 101860,  33121, 111201,  17620, 104770,  39926,  '
 '72315,\n'
 '          9174,     43, 108295, 114341, 104577,  18904,   5486,     48, '
 '108295,\n'
 '        105284,  99397, 122532,  30512,  20379,  78434,  10110,  27452,     '
 '25,\n'
 '           445,     16,  29342,     16, 123505, 113164, 102663,  84477,  '
 '16144,\n'
 '        

In [ ]:
a

### LLMの読み込み -> PEFT設定


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from torchinfo import summary

In [ ]:
# モデルにbatchを通して、出力を確認
# 4サンプルだけ取得
batch = {k: v[:4] for k, v in batch.items()}  # 4サンプルに制限
logger.info(f"Batch size: {len(batch['input_ids'])}")
logger.info(f"Input IDs shape: {batch['input_ids'].shape}")
logger.info(f"Attention mask shape: {batch['attention_mask'].shape}")
logger.info(f"Labels shape: {batch['labels'].shape}")

2025-06-07 01:00:45,625 : LLM_test : INFO : 4 : Batch size: 4
2025-06-07 01:00:45,627 : LLM_test : INFO : 5 : Input IDs shape: torch.Size([4, 3126])
2025-06-07 01:00:45,628 : LLM_test : INFO : 6 : Attention mask shape: torch.Size([4, 3126])
2025-06-07 01:00:45,629 : LLM_test : INFO : 7 : Labels shape: torch.Size([4, 3126])


In [ ]:
# "日本の首都はどこですか？"の質問に対する回答を生成
sample_text = "日本の首都はどこですか？"
inputs = tokenizer(
    sample_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=4096,
).to(model.device)
outputs = model(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    labels=inputs["input_ids"],  # ラベルとして同じ入力を使用
)

# 生成されたテキストをデコード
decoded_output = tokenizer.decode(
    outputs.logits.argmax(dim=-1)[0], skip_special_tokens=True
)
logger.info(f"Decoded output: {decoded_output}")

2025-06-07 01:17:12,011 : LLM_test : INFO : 20 : Decoded output: 、の伝相である、ですか？
日本



首あは

首日本日本日本日本日本日本日本の日日のの日本日本日本日本日本日本日本日本日本はは首の日本日本日本日本日本日本日本は
は日本日本日本日本日本日本のの日本日本日本

は
の日本日本は首首らはは日本日本日日日の日本日本のののの日本日本日本のの日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本
日本日本ら日本日本日日日のの日本日本日本日本日本日本日本日本日日日日日日年年日日日の日日日のの日本ののの日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本首の日本を日本日日日日日日日日日日本日本日本日日日日日日日日ののの日本日年://年日日年日日日の日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日本日.aa..://://年日日日のの日日日日日日年日日日日日年://://日日日の年.://://日日日日日日日の日日日日日日の日日日の日本日本日年日日年.aaaaaa://://年日://.://年日日日日日の日本のの日://://aaaaaaaaaaaa...://://年年://://aaaaaa...aaaaaaaaaaaaaaaaaaaaaa://://://aaaaaaaa日aa://aa://aaaa://aaaaaa..://.://日日日日日日日日日日日日日日日日年.年日日日日本日日のの日本日本日日年日日日日日日日日日本日本日本日日本日本日aaaaaaaaaa://://://://aaaa.aaaaaaaa.://年://://://://://://.aaaaaaaaaaaaaaaaaaaaaaaa://://aaaaaaaaaa://aaaaaaaa://日://aa://.aaaaaaaaaa....://://://://.aaaa://://://年日日の日本日本日本日本日本ののののの日日日日日日日日日日日日年年日日日日日本日年年年年://.://://://://://..aa.://日年aaaaaaaaaaaaaaaaaaaaaaaa://aaaaaa.

In [ ]:
# "日本の首都はどこですか？"の質問に対する回答を生成
sample_text = "日本の首都はどこですか？"
inputs = tokenizer(
    sample_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=4096,
).to(model.device)
# ⚠️ ラベルは `generate()` には不要。除去する！
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=64,  # <- 出力長のみを制限するのが推奨
    do_sample=False,  # or True (生成アルゴリズム)
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
logger.info(f"Decoded output: {decoded_output}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
2025-06-07 01:34:18,333 : LLM_test : INFO : 21 : Decoded output: 日本の首都はどこですか？assistant

日本の首都は東京都です。


In [ ]:
# モデルを評価モードに設定
model.eval()
# モデルの出力を取得
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=64,  # <- 出力長のみを制限するのが推奨
        do_sample=False,  # or True (生成アルゴリズム)
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# モデルの出力をデコードして、プロンプトを確認
decoded_output = tokenizer.batch_decode(
    outputs, skip_special_tokens=True
)  # outputsは生成されたトークンのテンソル
for item in decoded_output:
    logger.info(f"Decoded output item: {item}")
# logger.info(f"Decoded output: {decoded_output}")

2025-06-07 01:48:40,207 : LLM_test : INFO : 19 : Decoded output item: 日本の首都はどこですか？assistant

日本の首都は東京都です。


In [ ]:
# ① LoRA設定
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",  # Self-Attention系
        "gate_proj",
        "up_proj",
        "down_proj",  # MLP（FFN）系
    ],
)


In [ ]:
model.enable_input_require_grads()  #! 追加::入力テンソルに勾配を流せる状態を強制する安全スイッチ
model = get_peft_model(model, peft_config)

In [ ]:
logger.info(f"Trainable parameters:")
model.print_trainable_parameters()
logger.info("✅ LoRA has been successfully applied to the model.")
logger.debug(summary(model))

2025-06-07 01:51:15,966 : LLM_test : INFO : 1 : Trainable parameters:
2025-06-07 01:51:15,978 : LLM_test : INFO : 3 : ✅ LoRA has been successfully applied to the model.


trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


2025-06-07 01:51:16,419 : LLM_test : DEBUG : 4 : ==========================================================================================
Layer (type:depth-idx)                                            Param #
PeftModelForCausalLM                                              --
├─LoraModel: 1-1                                                  --
│    └─LlamaForCausalLM: 2-1                                      --
│    │    └─LlamaModel: 3-1                                       7,525,896,192
│    │    └─Linear: 3-2                                           (525,336,576)
Total params: 8,051,232,768
Trainable params: 20,971,520
Non-trainable params: 8,030,261,248


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
# モデルを評価モードに設定
model.eval()
# モデルの出力を取得
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=64,  # <- 出力長のみを制限するのが推奨
        do_sample=False,  # or True (生成アルゴリズム)
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# モデルの出力をデコードして、プロンプトを確認
decoded_output = tokenizer.batch_decode(
    outputs, skip_special_tokens=True
)  # outputsは生成されたトークンのテンソル
for item in decoded_output:
    logger.info(f"Decoded output item: {item}")
# logger.info(f"Decoded output: {decoded_output}")

2025-06-06 19:09:24,912 : LLM_test : INFO : 12 : Logits shape: torch.Size([4, 1041, 128256])
2025-06-06 19:09:24,913 : LLM_test : INFO : 13 : Loss: 1.5673637390136719
2025-06-06 19:09:25,176 : LLM_test : INFO : 16 : Decoded output: ['、]のなたは、生研究で、学生の質長を評する権割を担っています。学生の、す学生の成義ノアンケートの分析み、学生績を決定評に並、B、C、D、Fの5ずれかに評類しますください。\n\n講ecture、義の数T は質問のです示します。例： L1Q1）。Qケートの内容問とは以下Q1: 本の講は理解でりに言葉で説明してくださいてください。\nQ。\n\nアンは以下 の学生、回答を。\n\n以下席は、、B、C、D、F のいずれかを記むください。\n\n成ケートのはL1-Q1: 自回講学校の数学くは、系系の指り方放を共目カをと文やの、年数ややの表っているたいするがA01-Q1: 文記、回の、進数で表現います、データで特くからをつけるられる方法するくいろな方法夫がされている。\nL03-Q1: 二ータの表みりを合を、隔を防見する修正する。\nどうるくる。\nータの二進のパ子重まものックで位で管理信。\n�違いが発ぐために、々を増くするおる。\nL04-Q1: デ\nL05-Q1: デンピューターは�イエンスの、機速御するり、学プである。L06-Q1: コ\nL07-Q1: デフトやいていえばいろな方法がある学えるりL08-Q1: NaN\nL09-Q1: コータの重類や方法を、の機の学用方法をL10-Q1: NaNに電子の二造化データと非構造化データの分類られる\n\n、中でも非構造化データはAI類されるものは多析ははAIターン認識のAIれる方法の分類する識識する方法方法が二をL11-Q1: NaNータの集の方法と、々あるがことをと々情報の守する�つデータ々の情報を集用する方法をという\nL12-Q1: NaN\nL13-Q1: データの情報の整視化する方法、をれのような方法な方法方法する視化するいのかを可さんの方法フの種類